In [1]:
import findspark
findspark.init('C:/Users/gaborstefanics/Documents/GitHub/learnPySark/spark/spark-2.4.4-bin-hadoop2.7')

#create spark session
from pyspark.sql import SparkSession

spark=SparkSession.builder.appName('nlp').getOrCreate()

In [2]:
df=spark.createDataFrame([(1,'I really liked this movie'),
                         (2,'I would recommend this movie to my friends'),
                         (3,'movie was alright but acting was horrible'),
                         (4,'I am never watching that movie ever again')],
                        ['user_id','review'])

In [3]:
df.show(5,False)

+-------+------------------------------------------+
|user_id|review                                    |
+-------+------------------------------------------+
|1      |I really liked this movie                 |
|2      |I would recommend this movie to my friends|
|3      |movie was alright but acting was horrible |
|4      |I am never watching that movie ever again |
+-------+------------------------------------------+



In [4]:
# Tokenization

In [5]:
from pyspark.ml.feature import Tokenizer

In [6]:
tokenization=Tokenizer(inputCol='review',outputCol='tokens')

In [7]:
tokenized_df=tokenization.transform(df)

In [8]:
tokenized_df.show(4,False)

+-------+------------------------------------------+---------------------------------------------------+
|user_id|review                                    |tokens                                             |
+-------+------------------------------------------+---------------------------------------------------+
|1      |I really liked this movie                 |[i, really, liked, this, movie]                    |
|2      |I would recommend this movie to my friends|[i, would, recommend, this, movie, to, my, friends]|
|3      |movie was alright but acting was horrible |[movie, was, alright, but, acting, was, horrible]  |
|4      |I am never watching that movie ever again |[i, am, never, watching, that, movie, ever, again] |
+-------+------------------------------------------+---------------------------------------------------+



In [9]:
# stopwords removal 

In [10]:
from pyspark.ml.feature import StopWordsRemover

In [11]:
stopword_removal=StopWordsRemover(inputCol='tokens',outputCol='refined_tokens')

In [12]:
refined_df=stopword_removal.transform(tokenized_df)

In [13]:
refined_df.select(['user_id','tokens','refined_tokens']).show(10,False)

+-------+---------------------------------------------------+----------------------------------+
|user_id|tokens                                             |refined_tokens                    |
+-------+---------------------------------------------------+----------------------------------+
|1      |[i, really, liked, this, movie]                    |[really, liked, movie]            |
|2      |[i, would, recommend, this, movie, to, my, friends]|[recommend, movie, friends]       |
|3      |[movie, was, alright, but, acting, was, horrible]  |[movie, alright, acting, horrible]|
|4      |[i, am, never, watching, that, movie, ever, again] |[never, watching, movie, ever]    |
+-------+---------------------------------------------------+----------------------------------+



In [ ]:
# Count Vectorizer

In [14]:
from pyspark.ml.feature import CountVectorizer

In [15]:
count_vec=CountVectorizer(inputCol='refined_tokens',outputCol='features')

In [16]:
cv_df=count_vec.fit(refined_df).transform(refined_df)

In [17]:
cv_df.select(['user_id','refined_tokens','features']).show(4,False)

+-------+----------------------------------+--------------------------------+
|user_id|refined_tokens                    |features                        |
+-------+----------------------------------+--------------------------------+
|1      |[really, liked, movie]            |(11,[0,6,9],[1.0,1.0,1.0])      |
|2      |[recommend, movie, friends]       |(11,[0,3,10],[1.0,1.0,1.0])     |
|3      |[movie, alright, acting, horrible]|(11,[0,1,2,8],[1.0,1.0,1.0,1.0])|
|4      |[never, watching, movie, ever]    |(11,[0,4,5,7],[1.0,1.0,1.0,1.0])|
+-------+----------------------------------+--------------------------------+



In [18]:
count_vec.fit(refined_df).vocabulary

[u'movie',
 u'recommend',
 u'never',
 u'really',
 u'watching',
 u'acting',
 u'liked',
 u'friends',
 u'ever',
 u'horrible',
 u'alright']

In [ ]:
#Tf-idf

In [19]:
from pyspark.ml.feature import HashingTF,IDF

In [20]:
hashing_vec=HashingTF(inputCol='refined_tokens',outputCol='tf_features')

In [21]:
hashing_df=hashing_vec.transform(refined_df)

In [22]:
hashing_df.select(['user_id','refined_tokens','tf_features']).show(4,False)

+-------+----------------------------------+-------------------------------------------------------+
|user_id|refined_tokens                    |tf_features                                            |
+-------+----------------------------------+-------------------------------------------------------+
|1      |[really, liked, movie]            |(262144,[14,32675,155321],[1.0,1.0,1.0])               |
|2      |[recommend, movie, friends]       |(262144,[129613,155321,222394],[1.0,1.0,1.0])          |
|3      |[movie, alright, acting, horrible]|(262144,[80824,155321,236263,240286],[1.0,1.0,1.0,1.0])|
|4      |[never, watching, movie, ever]    |(262144,[63139,155321,203802,245806],[1.0,1.0,1.0,1.0])|
+-------+----------------------------------+-------------------------------------------------------+



In [23]:
tf_idf_vec=IDF(inputCol='tf_features',outputCol='tf_idf_features')

In [24]:
tf_idf_df=tf_idf_vec.fit(hashing_df).transform(hashing_df)

In [25]:
tf_idf_df.select(['user_id','tf_idf_features']).show(4,False)

+-------+----------------------------------------------------------------------------------------------------+
|user_id|tf_idf_features                                                                                     |
+-------+----------------------------------------------------------------------------------------------------+
|1      |(262144,[14,32675,155321],[0.9162907318741551,0.9162907318741551,0.0])                              |
|2      |(262144,[129613,155321,222394],[0.9162907318741551,0.0,0.9162907318741551])                         |
|3      |(262144,[80824,155321,236263,240286],[0.9162907318741551,0.0,0.9162907318741551,0.9162907318741551])|
|4      |(262144,[63139,155321,203802,245806],[0.9162907318741551,0.0,0.9162907318741551,0.9162907318741551])|
+-------+----------------------------------------------------------------------------------------------------+



In [ ]:
# Classification 

In [26]:
text_df=spark.read.csv('Movie_reviews.csv',inferSchema=True,header=True,sep=',')

In [27]:
text_df.printSchema()

root
 |-- Review: string (nullable = true)
 |-- Sentiment: string (nullable = true)



In [28]:
text_df.count()

7087

In [29]:
from pyspark.sql.functions import rand 

In [30]:
text_df.orderBy(rand()).show(10,False)

+------------------------------------------------------------------------+---------+
|Review                                                                  |Sentiment|
+------------------------------------------------------------------------+---------+
|Oh, and Brokeback Mountain is a TERRIBLE movie...                       |0        |
|Ok brokeback mountain is such a horrible movie.                         |0        |
|Combining the opinion / review from Gary and Gin Zen, The Da Vinci Code |0        |
|Brokeback Mountain was so awesome.                                      |1        |
|Come out of the closet and please stop making those terrible Mission Imp|0        |
|I want to be here because I love Harry Potter, and I really want a place|1        |
|Harry Potter dragged Draco Malfoy ’ s trousers down past his hips and   |0        |
|I also disliked Brokeback Mountain.                                     |0        |
|ok it's time to update wow haven't updated for a long time ok so

In [31]:
text_df=text_df.filter(((text_df.Sentiment =='1') | (text_df.Sentiment =='0')))

In [32]:
text_df.count()

6990

In [33]:
text_df.groupBy('Sentiment').count().show()

+---------+-----+
|Sentiment|count|
+---------+-----+
|        0| 3081|
|        1| 3909|
+---------+-----+



In [34]:
text_df.printSchema()

root
 |-- Review: string (nullable = true)
 |-- Sentiment: string (nullable = true)



In [35]:
text_df = text_df.withColumn("Label", text_df.Sentiment.cast('float')).drop('Sentiment')

In [36]:
text_df.orderBy(rand()).show(10,False)

+------------------------------------------------------------------------+-----+
|Review                                                                  |Label|
+------------------------------------------------------------------------+-----+
|I like Mission Impossible movies because you never know who's on the rig|1.0  |
|I love The Da Vinci Code...                                             |1.0  |
|I think the Academy was too chicken to give the Best Picture to Brokebac|0.0  |
|I hate Harry Potter and all things related to it..                      |0.0  |
|i heard da vinci code sucked soo much only 2.5 stars:                   |0.0  |
|The Da Vinci Code was awesome, I can't wait to read it...               |1.0  |
|It stayed on the whole way there and the whole way back.-Jill is staying|0.0  |
|Oh, and Brokeback Mountain is a TERRIBLE movie...                       |0.0  |
|Is it just me, or does Harry Potter suck?...                            |0.0  |
|Mission Impossible three su

In [37]:
text_df.groupBy('label').count().show()

+-----+-----+
|label|count|
+-----+-----+
|  1.0| 3909|
|  0.0| 3081|
+-----+-----+



In [38]:
# Add length to the dataframe
from pyspark.sql.functions import length

In [39]:
text_df=text_df.withColumn('length',length(text_df['Review']))

In [40]:
text_df.orderBy(rand()).show(10,False)

+-----------------------------------------------------------------------+-----+------+
|Review                                                                 |Label|length|
+-----------------------------------------------------------------------+-----+------+
|da vinci code sucks...                                                 |0.0  |22    |
|So as felicia's mom is cleaning the table, felicia grabs my keys and we|1.0  |71    |
|I absolutely LOVE Harry Potter, as you can tell already.               |1.0  |56    |
|I love Harry Potter..                                                  |1.0  |21    |
|Brokeback mountain was beautiful...                                    |1.0  |35    |
|Brokeback Mountain is fucking horrible..                               |0.0  |40    |
|Brokeback Mountain was an AWESOME movie.                               |1.0  |40    |
|The Da Vinci Code was awesome, I can't wait to read it...              |1.0  |57    |
|"Anyway, thats why I love "" Brokeback Mou

In [41]:
text_df.groupBy('Label').agg({'Length':'mean'}).show()

+-----+-----------------+
|Label|      avg(Length)|
+-----+-----------------+
|  1.0|47.61882834484523|
|  0.0|50.95845504706264|
+-----+-----------------+



In [42]:
# Data Cleaning

In [43]:
tokenization=Tokenizer(inputCol='Review',outputCol='tokens')

In [44]:
tokenized_df=tokenization.transform(text_df)

In [45]:
tokenized_df.show()

+--------------------+-----+------+--------------------+
|              Review|Label|length|              tokens|
+--------------------+-----+------+--------------------+
|The Da Vinci Code...|  1.0|    39|[the, da, vinci, ...|
|this was the firs...|  1.0|    72|[this, was, the, ...|
|i liked the Da Vi...|  1.0|    32|[i, liked, the, d...|
|i liked the Da Vi...|  1.0|    32|[i, liked, the, d...|
|I liked the Da Vi...|  1.0|    72|[i, liked, the, d...|
|that's not even a...|  1.0|    72|[that's, not, eve...|
|I loved the Da Vi...|  1.0|    72|[i, loved, the, d...|
|i thought da vinc...|  1.0|    57|[i, thought, da, ...|
|The Da Vinci Code...|  1.0|    45|[the, da, vinci, ...|
|I thought the Da ...|  1.0|    51|[i, thought, the,...|
|The Da Vinci Code...|  1.0|    68|[the, da, vinci, ...|
|The Da Vinci Code...|  1.0|    62|[the, da, vinci, ...|
|then I turn on th...|  1.0|    66|[then, i, turn, o...|
|The Da Vinci Code...|  1.0|    34|[the, da, vinci, ...|
|i love da vinci c...|  1.0|   

In [46]:
stopword_removal=StopWordsRemover(inputCol='tokens',outputCol='refined_tokens')

In [47]:
refined_text_df=stopword_removal.transform(tokenized_df)

In [48]:
refined_text_df.show()

+--------------------+-----+------+--------------------+--------------------+
|              Review|Label|length|              tokens|      refined_tokens|
+--------------------+-----+------+--------------------+--------------------+
|The Da Vinci Code...|  1.0|    39|[the, da, vinci, ...|[da, vinci, code,...|
|this was the firs...|  1.0|    72|[this, was, the, ...|[first, clive, cu...|
|i liked the Da Vi...|  1.0|    32|[i, liked, the, d...|[liked, da, vinci...|
|i liked the Da Vi...|  1.0|    32|[i, liked, the, d...|[liked, da, vinci...|
|I liked the Da Vi...|  1.0|    72|[i, liked, the, d...|[liked, da, vinci...|
|that's not even a...|  1.0|    72|[that's, not, eve...|[even, exaggerati...|
|I loved the Da Vi...|  1.0|    72|[i, loved, the, d...|[loved, da, vinci...|
|i thought da vinc...|  1.0|    57|[i, thought, da, ...|[thought, da, vin...|
|The Da Vinci Code...|  1.0|    45|[the, da, vinci, ...|[da, vinci, code,...|
|I thought the Da ...|  1.0|    51|[i, thought, the,...|[thought

In [49]:
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import *

In [50]:
len_udf = udf(lambda s: len(s), IntegerType())

refined_text_df = refined_text_df.withColumn("token_count", len_udf(col('refined_tokens')))


In [51]:
refined_text_df.orderBy(rand()).show(10)

+--------------------+-----+------+--------------------+--------------------+-----------+
|              Review|Label|length|              tokens|      refined_tokens|token_count|
+--------------------+-----+------+--------------------+--------------------+-----------+
|, she helped me b...|  0.0|    72|[,, she, helped, ...|[,, helped, bobby...|          8|
|I wanted desperat...|  1.0|    57|[i, wanted, despe...|[wanted, desperat...|          7|
|Because I would l...|  1.0|    72|[because, i, woul...|[like, make, frie...|          6|
|Combining the opi...|  0.0|    71|[combining, the, ...|[combining, opini...|         10|
|The Da Vinci Code...|  1.0|    30|[the, da, vinci, ...|[da, vinci, code,...|          4|
|Oh, and Brokeback...|  0.0|    48|[oh,, and, brokeb...|[oh,, brokeback, ...|          5|
|I hate Harry Pott...|  0.0|    71|[i, hate, harry, ...|[hate, harry, pot...|          7|
|Then snuck into B...|  0.0|    72|[then, snuck, int...|[snuck, brokeback...|          5|
|"I liked 

In [52]:
count_vec=CountVectorizer(inputCol='refined_tokens',outputCol='features')

In [53]:
cv_text_df=count_vec.fit(refined_text_df).transform(refined_text_df)

In [54]:
cv_text_df.select(['refined_tokens','token_count','features','Label']).show(10)

+--------------------+-----------+--------------------+-----+
|      refined_tokens|token_count|            features|Label|
+--------------------+-----------+--------------------+-----+
|[da, vinci, code,...|          5|(2302,[0,1,4,43,2...|  1.0|
|[first, clive, cu...|          9|(2302,[11,51,229,...|  1.0|
|[liked, da, vinci...|          5|(2302,[0,1,4,53,3...|  1.0|
|[liked, da, vinci...|          5|(2302,[0,1,4,53,3...|  1.0|
|[liked, da, vinci...|          8|(2302,[0,1,4,53,6...|  1.0|
|[even, exaggerati...|          6|(2302,[46,229,271...|  1.0|
|[loved, da, vinci...|          8|(2302,[0,1,22,30,...|  1.0|
|[thought, da, vin...|          7|(2302,[0,1,4,228,...|  1.0|
|[da, vinci, code,...|          6|(2302,[0,1,4,33,2...|  1.0|
|[thought, da, vin...|          7|(2302,[0,1,4,223,...|  1.0|
+--------------------+-----------+--------------------+-----+
only showing top 10 rows



In [55]:
#select data for building model
model_text_df=cv_text_df.select(['features','token_count','Label'])

In [56]:
from pyspark.ml.feature import VectorAssembler

In [57]:
df_assembler = VectorAssembler(inputCols=['features','token_count'],outputCol='features_vec')
model_text_df = df_assembler.transform(model_text_df)

In [58]:
model_text_df.printSchema()

root
 |-- features: vector (nullable = true)
 |-- token_count: integer (nullable = true)
 |-- Label: float (nullable = true)
 |-- features_vec: vector (nullable = true)



In [59]:
from pyspark.ml.classification import LogisticRegression

In [60]:
#split the data 
training_df,test_df=model_text_df.randomSplit([0.75,0.25])

In [61]:
training_df.groupBy('Label').count().show()

+-----+-----+
|Label|count|
+-----+-----+
|  1.0| 2947|
|  0.0| 2300|
+-----+-----+



In [62]:
test_df.groupBy('Label').count().show()

+-----+-----+
|Label|count|
+-----+-----+
|  1.0|  962|
|  0.0|  781|
+-----+-----+



In [63]:
log_reg=LogisticRegression(featuresCol='features_vec',labelCol='Label').fit(training_df)

In [64]:
results=log_reg.evaluate(test_df).predictions

In [65]:
results.show()

+--------------------+-----------+-----+--------------------+--------------------+--------------------+----------+
|            features|token_count|Label|        features_vec|       rawPrediction|         probability|prediction|
+--------------------+-----------+-----+--------------------+--------------------+--------------------+----------+
|(2302,[0,1,4,10,2...|         11|  0.0|(2303,[0,1,4,10,2...|[18.104463387316,...|[0.99999998628071...|       0.0|
|(2302,[0,1,4,10,2...|          7|  0.0|(2303,[0,1,4,10,2...|[23.8507885831079...|[0.99999999995617...|       0.0|
|(2302,[0,1,4,11,1...|          6|  0.0|(2303,[0,1,4,11,1...|[19.6201310554783...|[0.99999999698640...|       0.0|
|(2302,[0,1,4,12,2...|          7|  1.0|(2303,[0,1,4,12,2...|[-19.133461366005...|[4.90279017261056...|       1.0|
|(2302,[0,1,4,12,3...|          5|  1.0|(2303,[0,1,4,12,3...|[-19.742601804572...|[2.66622610758610...|       1.0|
|(2302,[0,1,4,12,3...|          5|  1.0|(2303,[0,1,4,12,3...|[-19.742601804572..

In [66]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator


In [67]:
#confusion matrix
true_postives = results[(results.Label == 1) & (results.prediction == 1)].count()
true_negatives = results[(results.Label == 0) & (results.prediction == 0)].count()
false_positives = results[(results.Label == 0) & (results.prediction == 1)].count()
false_negatives = results[(results.Label == 1) & (results.prediction == 0)].count()

In [68]:
recall = float(true_postives)/(true_postives + false_negatives)
print(recall)

0.982328482328


In [69]:
precision = float(true_postives) / (true_postives + false_positives)
print(precision)

0.966257668712


In [71]:
accuracy=float(true_postives+true_negatives) /(results.count())
print(accuracy)

0.971313826736
